### Routing - Agentic Design Pattern

In [ ]:
import os

from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import colored
from dotenv import load_dotenv
from pydantic import BaseModel

import json
from rich.pretty import pprint
from uuid import uuid4

In [ ]:
load_dotenv()

In [ ]:
host = os.environ["TOGETHER_URL"]
together_api_key = os.environ["TOGETHER_API_KEY"]

client = LlamaStackClient(
    base_url=host,
    provider_data={
        "tavily_search_api_key": os.getenv("TAVILY_SEARCH_API_KEY"),
        "together_api_key": together_api_key,
    },
)

In [ ]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are a helpful assistant.",
    sampling_params={
        "strategy": {
            "type": "top_p",
            "temperature": 1.0,
            "top_p": 0.9},
    },
)

In [ ]:
billing_agent_config = {
    **base_agent_config,
    "instructions": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.
    """,
}

technical_agent_config = {
    **base_agent_config,
    "instructions": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.
    """,
}

account_agent_config = {
    **base_agent_config,
    "instructions": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.
    """,
}

product_agent_config = {
    **base_agent_config,
    "instructions": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone.
    """,
}

In [ ]:
specialized_agents = {
    "billing": Agent(client, **billing_agent_config),
    "technical": Agent(client, **technical_agent_config),
    "account": Agent(client, **account_agent_config),
    "product": Agent(client, **product_agent_config),
}

In [ ]:
class OutputSchema(BaseModel):
    reasoning: str
    support_team: str


In [ ]:
routing_agent_config = {
    **base_agent_config,
    "instructions": f"""You are a routing agent. Analyze the user's input and select the most appropriate support team from these options: 

    {list(specialized_agents.keys())}

    Return the name of the support team in JSON format.

    First explain your reasoning, then provide your selection in this JSON format: 
    {{
        "reasoning": "<your explanation>",
        "support_team": "<support team name>"
    }}

    Note the support team name can only be one of the following: {specialized_agents.keys()}
    """,
    "response_format": {
        "type": "json_schema",
        "json_schema": OutputSchema.model_json_schema()
    }
}


In [ ]:
routing_agent = Agent(client, **routing_agent_config)


In [ ]:
routing_agent_session_id = routing_agent.create_session(session_name=f"routing_agent_{uuid4()}")
specialized_agents_session_ids = {
    "billing": specialized_agents["billing"].create_session(session_name=f"billing_agent_{uuid4()}"),
    "technical": specialized_agents["technical"].create_session(session_name=f"technical_agent_{uuid4()}"),
    "account": specialized_agents["account"].create_session(session_name=f"account_agent_{uuid4()}"),
    "product": specialized_agents["product"].create_session(session_name=f"product_agent_{uuid4()}"),
}


In [ ]:
def process_user_query(query):
    routing_response = routing_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        session_id=routing_agent_session_id,
        stream=False,
    )
    try:
        routing_result = json.loads(routing_response.output_message.content)
        
        pprint(f"🔀 [cyan] Routing Result: {routing_result['reasoning']} [/cyan]")
        pprint(f"🔀 [cyan] Routing to {routing_result['support_team']}... [/cyan]")

        return specialized_agents[routing_result["support_team"]].create_turn(
            messages=[
                {"role": "user", "content": query}
            ],
            session_id=specialized_agents_session_ids[routing_result["support_team"]],
            stream=False,
        )
    except json.JSONDecodeError:
        print("Error: Invalid JSON response from routing agent")
        return None

In [ ]:
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",

    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",

    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

In [ ]:
for i, ticket in enumerate(tickets):
    print(f"========= Processing ticket {i+1}: =========")
    response = process_user_query(ticket)
    print(response.output_message.content)
    print("\n")

In [ ]:
routing_agent_session = client.agents.session.retrieve(
    session_id=routing_agent_session_id, agent_id=routing_agent.agent_id)

print("Routing Agent Session:")
pprint(routing_agent_session.to_dict())

for specialized_agent_type, specialized_agent in specialized_agents.items():
    specialized_agent_session = client.agents.session.retrieve(session_id=specialized_agent.session_id, agent_id=specialized_agent.agent_id)
    
    print(f"Specialized Agent {specialized_agent_type} Session:")
    pprint(specialized_agent_session.to_dict())